# Загрузка библиотек

In [4]:
import numpy as np
import pandas as pd
import typing

## Функция сигнала ошибки

In [2]:
def error_signal(desired, actual):
    return desired - actual

## Общая энергия ошибки

ej(n) = dj(n) - yj(n)

In [3]:
def error_energy(desired, actual):
    return 0.5 * np.sum(np.float_power(error_signal(desired, actual), 2))

## Функция стоимости - мера эффективности обучения

In [4]:
def cost_function(desired, actual):
    return error_energy(desired, actual)

## Сигмоида

In [5]:
def diff_sigmoid(S: float, a: float=1.0):
    a * np.exp(-S * a) * sigmoid(S, a) ** 2

Функция сигмоиды:
![sigmoid](sigmoid.png)

In [6]:
def sigmoid(S, a: float=1.0):
    return np.div(1.0, np.sum(np.exp(np.mul(S, -a)), 1))

## Сумматор

In [7]:
def summator(w, x):
    return np.dot(w, x)

## Выходной сигнал

In [8]:
def output_signal(x, w):
    return sigmoid(summator(x, w))

## Среднеквадратическая ошибка

def mse(error):
    np.sum(np.float_power(error, 2))

# Заполнение весов

Т.к. без понятия, как подобрать начальный вес, воспользуемся рандомом в границах от -2 до 2. Можно от 0 до 1

In [9]:
def weight_matrix_init(neuorons_count: int, vector_size: int):
    matrix = np.empty([neuorons_count, vector_size], dtype=float)
    return matrix - (matrix // 2) * 2

# Нормализация данных

In [10]:
def data_normalise(data: pd.Series):
    return (data - data.min()) / (data.max() - data.min())

In [12]:
def runLayer(input_data, layer):
    return sigmoid(layer * input_data)


In [13]:
def direct_sequence(input_data, target):
    first_layer_res = runLayer(input_data, first_layer_weights)
    second_layer_res = runLayer(first_layer_res, second_layer_weights)
    return error_signal(target, second_layer_res), [first_layer_res, second_layer_res]

![title](local_grad_output.png)

In [14]:
def weight_output_delta(error_signal, layer_input):
    return error_signal * diff_sigmoid(layer_input)

![title](local_gard_hidden.png)

In [15]:
def weight_hidden_delta(layer_delta, layer_input, layer_weight):
    return diff_sigmoid(np.sum(layer_input)) * np.dot(layer_delta, layer_weight)

![title](delta_rule.png)

/\wji(n) = teta * delta(n) * yi(n)
teta = одна сотая, тысячная ... (на сколько мы строги в обучении ребёнка)
Есть два градиента: для выходного слоя, есть для скрытого слоя.

In [16]:
def delta_rule(delta, layer_output, tetta: float = 0.01):
    return tetta * pd.mul(delta, layer_output)

In [17]:
def back_propagation(error_sign, layers_inputs):
    output_delta = weight_output_delta(error_sign, layers_inputs[1])
    second_layer_delta = weight_hidden_delta(output_delta, layers_inputs[0])
    second_layer_weights = second_layer_weights + layers_inputs[2] * output_delta
    first_layer_weights = first_layer_weights + layers_inputs[1] * second_layer_delta

In [22]:
neurons_first_layer_count = 10
neurons_second_layer_count = 10
vector_size = 40
epochs = 10

### Инициализация весов слоёв

In [19]:
first_layer_weights = weight_matrix_init(neurons_first_layer_count, vector_size)
second_layer_weights = weight_matrix_init(neurons_second_layer_count, neurons_first_layer_count)

### Основной цикл

In [28]:
for epoch in range(epochs):
    for row_id, _ in kgf_df.iterrows():
        error_sign, layers_inputs = direct_sequence(
            target_data.iloc[row_id],
            features_data.iloc[row_id]
        )
        back_propagation(error_sign)

#print(mse)
#output mse in the file


ValueError: operands could not be broadcast together with shapes (10,40) (2,) 

# Разделение выборки на тестовую и обучающую

In [5]:
def split_dataset(df: pd.DataFrame, learn_percent: int, target_columns: list) -> typing.Tuple:
    if learn_percent < 0 or learn_percent > 100:# or learn_percent < 50:
        return None, None

    no_nans = df[(~np.isnan(df[target_columns]).any(axis=1)) & ~np.isnan(df[target_columns]).all(axis=1)]
    some_nans = df[(~np.isnan(df[target_columns])).any(axis=1) & (np.isnan(df[target_columns])).any(axis=1)]

    num_in_learn_no_nans = int(len(no_nans.index) / 100 * learn_percent)
    num_in_learn_some_nans = int(len(some_nans.index) / 100 * learn_percent)

    learn_no_nans = no_nans.sample(num_in_learn_no_nans)
    learn_some_nans = some_nans.sample(num_in_learn_some_nans)

    test_no_nans = no_nans.merge(learn_no_nans, how='left', indicator=True) \
        .query("_merge == 'left_only'").drop('_merge', axis=1)[no_nans.columns]
    test_some_nans = some_nans.merge(learn_some_nans, how='left', indicator=True) \
        .query("_merge == 'left_only'").drop('_merge', axis=1)[some_nans.columns]

    learn = pd.concat([learn_no_nans, learn_some_nans], ignore_index=True)
    test = pd.concat([test_no_nans, test_some_nans], ignore_index=True)

    return learn.sample(n=len(learn.index)).reset_index(drop=True), test

# Регрессионная модель на примере со скважинами

### Произведем разбивку на тестовую и обучающую

In [7]:
df = pd.read_csv("Очищенный датасет скважины.csv")
learn, test = split_dataset(df, learn_percent = 80, target_columns = df.columns[-2:])
print(len(df.columns))

22


Дифференцируем: dE(n) / dwij(n) = ej(n) * (-1) * phi'(vi(n)) * yj(n)
vi - сумма выходных сигналов нейрона

In [ ]:
def dE_dw():
    error_signal() * diff_sigmoid(summator(x,w)) * output_signal(x, w)

# Градиентный спуск

* Оригинальный - долго. Гоняет туда-сюда.
* Стохастический - за один прогон. То что запомнили за прямой прогон. Один раз туда, один раз прогнали и затем применили.
* Пакетный - берём не всю, а батчевый. Мы будем гнать не по 200, не по одной, а, допустим, по 10. И затем для пакета все это применим

Синопс, а не нейрон. Разные сущности.
Прогнали, запомнили. На какойто посчитали для входного, затем выходного. Применили веса.

# Регуляризация (по желанию)

Рекомендутся сделать элементарный стохастический град спуск. Считать по формулам со слайда

Сигмоида только с положительными. Гипер Тангенс - знак. ReLu про другое.
Главное, что число гиперпараметров сети должно было хватить, чтобы обобщить.
Нейронка не должна быть маленькой.
Два слоя по 50 нейронов. (Чувак показывал, всё получилось)
R2 score посчитать, MSE. Обобщаем на многомерный случай. ROC-AUC можно.

# 70% - 30%. Обучать на тестовой